In [11]:
import pandas as pd
import numpy as np
from typing import Dict, Tuple
import os
import pickle
global fold
from pathlib import Path
fold = "sample_election"

In [12]:
class Ballot(object):
    
    def __init__(self, votes: list):
        self.votes = [0] * len(votes)
        for i, v in enumerate(votes):
            self.votes[int(v) - 1] = i
        
    def vote(self) -> int:
        return self.votes[0]
    
    def vote_n(self, n: int) -> int:
        return self.votes[n]
    
    def runoff(self, n: int):
        self.votes.remove(n)

In [30]:
def load_absentee(path):
    """Loads in a Qualtrics spreadsheet and returns its info.
    
    Accepts a path to a spreadsheet, and loads it in. It will then return
    a dictionary associating vote numbers to names, and an array with votes.
    TODO: rewrite without numpy or pandas
    """
    
    elections = [("show", 3), ("spicy", 4)]
    
    # reads in data, and creates array of votes
    data = np.array(pd.read_csv(path))
    votes = data[2:, 17:]
        
    global fold
        
    os.mkdir(fold)
    os.chdir(fold)
    sofar = 0
    
    for title, length in elections:
        os.mkdir(title)
        os.chdir(title)
    
        with open("absentee.dat", "wb") as file:
            
            savable = votes[:, sofar: sofar + length]
            
            sofar += length
            
            pickle.dump(savable, file)
            os.chdir("..")
            
    os.chdir("..")
    
# load_absentee("Absentee Test_April 22, 2023_14.52.csv")

In [31]:
def load_csv(path: os.PathLike) -> Tuple[Dict[int, str], np.array]:
    """Loads in a Qualtrics spreadsheet and returns its info.
    
    Accepts a path to a spreadsheet, and loads it in. It will then return
    a dictionary associating vote numbers to names, and an array with votes.
    TODO: rewrite without numpy or pandas
    """
    # reads in data, and creates array of votes
    data = np.array(pd.read_csv(path))
    votes = data[2:, 17:]
    
    # creates names dictionary
    temp_names = data[0][17:]
    names = {}
    for i, val in enumerate(temp_names):
        names[i] = val.split(" - ")[-1]
    
    return names, votes

In [50]:
def main(pathname):
    
    pathname = Path(fold) / pathname
    
    with open(pathname / "absentee.dat", "rb") as f:
        absentees = pickle.load(f)
    
    files = pathname.glob("*.csv")
    
    for fname in files:
        names, votes = load_csv(fname)
        break
    
    ballots = []
    
    for v in votes:
        ballots.append(Ballot(v))
        ballots.append(Ballot(v))
        
    for a in absentees:
        ballots.append(Ballot(a))
        
    n = len(ballots)
        
    cast = {}
    for v in range(len(names)):
        cast[v] = 0
        
    flag = False
    
    while True:
        for b in ballots:
            cast[b.vote()] += 1
            
        min_opt = None
        min_votes = n 
        for option in cast:
            votes = cast[option]
            if votes > n/2:
                flag = True
                break
            elif votes < min_votes:
                min_opt = option
                min_votes = votes
        
        if flag: break
        
        for b in ballots:
            b.runoff(min_opt)
        cast.pop(min_opt)
        for i in cast:
            cast[i] = 0
            
        print(names[min_opt], "eliminated with", min_votes/n)
        
    print("selected", names[option])

In [51]:
main("show")

9 to 5 eliminated with 0.25
Once on this Island eliminated with 0.5
selected RENT
